In [1]:
from preprocess import *
from COLD import * 
from LBP import * 
from HOG import *
import pandas as pd
from classify import *


In [6]:
mainDataset=pd.read_csv('NonNormalized.csv')
featuresData=mainDataset.iloc[:,3:]
mean = featuresData.mean()
std = featuresData.std()

def predict(imgName, classifier,f):
    linesTable = []
    linesList = PreprocessImage(imgName,isPrediction=True)
    
    for line in linesList:
        coldHist = ColdFeature(line)
        lbpHist = extract_lbp(line)
        hogHist = calculateHOG(line)
        dataValues = [imgName]
        dataValues.extend(coldHist)
        dataValues.extend(lbpHist)
        dataValues.extend(hogHist)
        linesTable.append(dataValues)

    COLDLabels = ['COLD_'+str(i) for i in range(100)]
    LBPLabels = ['LBP_'+str(i) for i in range(256)]
    HOGLabels = ['HOG_'+str(i) for i in range(144)]
    labels = ['ImgName']
    labels.extend(COLDLabels)
    labels.extend(LBPLabels)
    labels.extend(HOGLabels)

    linesTable = np.array(linesTable)

    if(len(linesTable)>0):

        df = pd.DataFrame(data=linesTable,columns=labels)

        for column in df: 
            if column != 'Gender' and column != 'ImgName':
                df[column] = df[column].astype(float)

        #remove columns not found in the normal dataset
        for col in df:
            if not col in mainDataset.columns:
                df=df.drop(columns=[col])

        #Calculate standard normalization
        df.iloc[:,1:] = df.iloc[:,1:] - mean / std

        x_test = np.array(df.iloc[:,1:88])

        #Classify here
        y_pred = classifier.predict(x_test)
        print(y_pred)
        y=np.round(np.mean(y_pred))
        f.write(str(int(y)))
        f.write('\n')

    else:
        ### NO LINES FOUND
        f.write('1 no lines')
        f.write('\n')



In [3]:
x_train, x_valid,y_train,y_valid=read_data('ShuffledData.csv')
print('output with svm with rbf kernel')
starttime=time.time()
accuracies=[]
cs=np.arange(0.1,5.1,0.1)
for i in cs:
    Classifier=classify(x_train,y_train,0,i)
    accuracies.append(test(Classifier,x_valid,y_valid))
c=cs[accuracies.index(max(accuracies))]

Classifier1=classify(x_train,y_train,0,c)
print('running time = ',time.time()-starttime,'sec')


# X_train, X_test,X_valid,y_valid, y_train, y_test = readData("Dataset.csv")
# estimator = get_best_parameter(X_train,y_train,X_valid,y_valid)
# regressor = get_Classification(X_train,y_train,estimator)

output with svm with rbf kernel
running time =  10.982125997543335 sec


In [7]:
f = open("result.txt", "w")
f2=open("time.txt","w")
        
### ENTER FOLDER NAME HERE
folderName = 'test'

for imgPath in os.listdir(folderName):
    starttime=time.time()
    predict(folderName+'/'+imgPath,Classifier1,f)
    endtime=time.time() 
    totaltime = endtime - starttime
    f2.write(str(round(totaltime, 2)))
    f2.write('\n')
f.close()
f2.close()


[0. 0. 0. 0.]
[0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 0.]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [12]:
arr = np.array([1,2,3,4,5])
X_train, X_test, y_train, y_test = train_test_split(arr, arr, test_size=0.4,random_state=0)




array([3, 1])